# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm
import torch_optimizer
#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import utils
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_model import EWCLoss
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
# utils.freeze_parameters(model)

## load config and set logger

In [5]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [6]:
# Setting seed
modeler.set_seed(42)

In [7]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
PREMODEL_ROOT = config['IO_OPTION']['PREMODEL_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_FISHER_DIR = OUTPUT_ROOT + '/fisher'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_FISHER_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp4/config.yaml'

In [8]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [9]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # source mode
    dev_train_paths = dev_train_source_paths + dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(dev_train_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # source mode
    add_dev_paths = add_dev_source_paths + add_dev_target_paths
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3026, source_size : 3105, target_size : 9
Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [10]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [11]:
import metrics

In [12]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('get fisher info mat')
    model = Model()
    premodel_path = f'{PREMODEL_ROOT}/{machine_types[0]}_model.pkl'
    pretrained_dict = torch.load(premodel_path)
    model.load_state_dict(pretrained_dict, strict=False)
    
    optimizer = torch_optimizer.RAdam(model.parameters())
    
    metric = metrics.ArcMarginProduct(1280, 6)
    #ewc_loss = EWCLoss(frozen_model, fisher, lambda_ewc=1e4)
    fisher = modeler.estimate_fisher(model,
                                     dataloaders_dict['train'],
                                     sample_size=len(dev_paths[machine_type]['train']),
                                     metric=metric,
                                     batch_size=64)
    
    fisher_out_path = f'{OUT_FISHER_DIR}/{machine_type}_fisher.pkl'
    pd.to_pickle(fisher, fisher_out_path)
    logger.info(f'SAVE SUCCESS : {fisher_out_path}')
    
    com.toc()
    return fisher

# run

In [13]:
for machine_type in machine_types:
    fisher = run(machine_type, dev_paths)

2021-09-28 15:08:22,222 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2021-09-28 15:08:22,222 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-28 15:08:22,227 - 00_train.py - INFO - get fisher info mat
 99%|█████████▉| 94/95 [01:23<00:00,  1.13it/s]
2021-09-28 15:09:49,550 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp4/fisher/fan_fisher.pkl
2021-09-28 15:09:49,552 - 00_train.py - INFO - TARGET MACHINE_TYPE: gearbox
2021-09-28 15:09:49,553 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-28 15:09:49,555 - 00_train.py - INFO - get fisher info mat


elapsed time: 88.358413696 [sec]


 99%|█████████▉| 95/96 [01:20<00:00,  1.17it/s]
2021-09-28 15:11:10,996 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp4/fisher/gearbox_fisher.pkl
2021-09-28 15:11:11,001 - 00_train.py - INFO - TARGET MACHINE_TYPE: pump
2021-09-28 15:11:11,001 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-28 15:11:11,004 - 00_train.py - INFO - get fisher info mat


elapsed time: 81.446130991 [sec]


 99%|█████████▉| 94/95 [01:14<00:00,  1.26it/s]
2021-09-28 15:12:26,235 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp4/fisher/pump_fisher.pkl
2021-09-28 15:12:26,239 - 00_train.py - INFO - TARGET MACHINE_TYPE: slider
2021-09-28 15:12:26,240 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-28 15:12:26,243 - 00_train.py - INFO - get fisher info mat


...

elapsed time: 62.305954695 [sec]


 99%|█████████▉| 94/95 [01:16<00:00,  1.23it/s]
2021-09-28 15:16:11,100 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp4/fisher/ToyTrain_fisher.pkl
2021-09-28 15:16:11,104 - 00_train.py - INFO - TARGET MACHINE_TYPE: valve
2021-09-28 15:16:11,118 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-28 15:16:11,121 - 00_train.py - INFO - get fisher info mat


elapsed time: 77.134254932 [sec]


 99%|█████████▉| 94/95 [01:25<00:00,  1.10it/s]
2021-09-28 15:17:37,385 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp4/fisher/valve_fisher.pkl


elapsed time: 86.283831835 [sec]


In [14]:
fisher

{'bn0.weight': tensor([0.0126, 0.0241, 0.0061, 0.0046, 0.0085, 0.0073, 0.0478, 0.0061, 0.0567,
         0.0071, 0.0045, 0.0024, 0.0039, 0.0038, 0.0105, 0.0131, 0.0199, 0.0035,
         0.0088, 0.0038, 0.0184, 0.0042, 0.0025, 0.0026, 0.0048, 0.0036, 0.0064,
         0.0126, 0.0086, 0.0053, 0.0024, 0.0017, 0.0085, 0.0010, 0.0044, 0.0259,
         0.0068, 0.0219, 0.0166, 0.0106, 0.0017, 0.0007, 0.0021, 0.0040, 0.0025,
         0.0009, 0.0009, 0.0006, 0.0022, 0.0004, 0.0057, 0.0005, 0.0011, 0.0003,
         0.0007, 0.0008, 0.0015, 0.0008, 0.0019, 0.0012, 0.0012, 0.0005, 0.0023,
         0.0031, 0.0014, 0.0009, 0.0085, 0.0078, 0.0053, 0.0005, 0.0058, 0.0018,
         0.0018, 0.0065, 0.0042, 0.0050, 0.0211, 0.0033, 0.0027, 0.0074, 0.0019,
         0.0008, 0.0046, 0.0014, 0.0087, 0.0042, 0.0019, 0.0020, 0.0020, 0.0013,
         0.0014, 0.0016, 0.0020, 0.0007, 0.0080, 0.0108, 0.0016, 0.0012, 0.0074,
         0.0052, 0.0023, 0.0022, 0.0013, 0.0013, 0.0056, 0.0016, 0.0021, 0.0037,
         0.007